In [ ]:
import networkx as nx

relato_graph = nx.read_graphml("../Graphs/relato_graph.graphml")
corr_graph = nx.read_graphml('../Graphs/pearson_correlation_full_graph.graphml')
threshold_graph = nx.read_graphml('../Graphs/pearson_correlation_threshold_graph.graphml')
mst_graph = nx.read_graphml('../Graphs/pearson_correlation_mst_graph.graphml')

In [ ]:
print("Nodes:")
for node_id, attributes in relato_graph.nodes(data=True):
    print(f"Node {node_id}: {attributes}")

print("\nEdges:")
for edge in relato_graph.edges(data=True):
    print(f"Edge {edge[:2]}: {edge[2]}")

In [ ]:
print("Nodes:")
for node_id, attributes in corr_graph.nodes(data=True):
    print(f"Node {node_id}: {attributes}")
print("\nEdges:")
for edge in corr_graph.edges(data=True):
    print(f"Edge {edge[:2]}: {edge[2]}")

In [ ]:
combined_corr_graph = nx.Graph()

combined_corr_graph.add_nodes_from(corr_graph.nodes(data=True))

for edge in corr_graph.edges(data=True):
    company1 = edge[0]
    company2 = edge[1]
    weight = edge[2]['weight']
    
    if relato_graph.has_edge(company1, company2):
        relationship_types = relato_graph[company1][company2]['types']
    else:
        relationship_types = ''

    combined_corr_graph.add_edge(company1, company2, weight=weight, types=[relationship_types])

for edge in combined_corr_graph.edges():
    if 'types' in combined_corr_graph[edge[0]][edge[1]]:
        combined_corr_graph[edge[0]][edge[1]]['types'] = ', '.join(combined_corr_graph[edge[0]][edge[1]]['types'])
 

In [ ]:
print(combined_corr_graph)

In [ ]:
print("Nodes:")
for node_id, attributes in combined_corr_graph.nodes(data=True):
    print(f"Node {node_id}: {attributes}")
print("\nEdges:")
for edge in combined_corr_graph.edges(data=True):
    print(f"Edge {edge[:2]}: {edge[2]}")

In [ ]:
edges = []
for edge in combined_corr_graph.edges(data=True):
    if edge[2]['types'] != '':
        edges.append(edge)

print(edges)
print(len(edges))

for edge in edges:
    print(edge[2])

In [ ]:
nodes = []
for relato_node in relato_graph.nodes(data=True):
    for corr_node in corr_graph.nodes(data=True):
        if relato_node[0] == corr_node[0]:
            nodes.append(relato_node[0])

nodes = list(set(nodes))
print(len(nodes))

In [ ]:
comb_graph = nx.Graph()

for node in nodes:
    comb_graph.add_node(node)
for edge in edges:
    comb_graph.add_edge(edge[0], edge[1], weight=edge[2]['weight'], types=edge[2]['types'])

print(comb_graph)

In [ ]:

import matplotlib.pyplot as plt

plt.figure(figsize=(15, 15))

pos = nx.spring_layout(comb_graph)
nx.draw_networkx(comb_graph, pos, with_labels=True, font_size=8, font_color="black", font_weight="bold", edge_color="gray", width=0.1)

# Show the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 15))

pos = nx.random_layout(comb_graph)
nx.draw_networkx(comb_graph, pos, with_labels=True, font_size=8, font_color="black", font_weight="bold", edge_color="gray", width=0.1)

# Show the plot
plt.show()

In [ ]:
threshold = 0.5
comb_threshold_graph = combined_corr_graph

remove_edges = [(u, v) for u, v, d in comb_threshold_graph.edges(data=True) if d['weight']<threshold]
comb_threshold_graph.remove_edges_from(remove_edges)

In [ ]:
print(comb_threshold_graph)

In [ ]:

import matplotlib.pyplot as plt

plt.figure(figsize=(15, 15))

pos = nx.random_layout(comb_threshold_graph)
nx.draw_networkx(comb_threshold_graph, pos, with_labels=True, font_size=8, font_color="black", font_weight="bold", edge_color="gray", width=0.1)

# Show the plot
plt.show()

In [ ]:
print("Nodes:")
for node_id, attributes in comb_threshold_graph.nodes(data=True):
    print(f"Node {node_id}: {attributes}")
print("\nEdges:")
for edge in comb_threshold_graph.edges(data=True):
    print(f"Edge {edge[:2]}: {edge[2]}")

In [ ]:
import os
from dotenv import load_dotenv
import pandas as pd

load_dotenv()
stock_returns_path = os.getenv('STOCK_RETURNS_5YRS_FILE_PATH')

df = pd.read_csv('D:\MANU\Datasets\yfinance_stock_returns.csv')
df.set_index('Date', inplace=True)

df.head()

In [ ]:
# Add daily log returns as features to every node (company)
for company in comb_threshold_graph.nodes:
    company_data = df[df['symbol'] == company]
    log_returns_dict = dict(zip(company_data.index, company_data['Adj Close']))
    nx.set_node_attributes(comb_threshold_graph, {company: log_returns_dict}, name='Adj Close')


In [ ]:
import json

# Convert log_returns from dictionary to string
for company in comb_threshold_graph.nodes:
    log_returns_dict = nx.get_node_attributes(comb_threshold_graph, 'Adj Close').get(company, {})
    log_returns_str = json.dumps(log_returns_dict)
    nx.set_node_attributes(comb_threshold_graph, {company: log_returns_str}, name='Adj Close')


In [ ]:
print("Nodes:")
for node_id, attributes in comb_threshold_graph.nodes(data=True):
    print(f"Node {node_id}: {attributes}")
print("\nEdges:")
for edge in comb_threshold_graph.edges(data=True):
    print(f"Edge {edge[:2]}: {edge[2]}")

In [ ]:
nx.write_graphml(comb_threshold_graph, '../Graphs/combined_threshold_graph.graphml')